In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install faiss-cpu

In [ ]:
import pandas as pd

#Read data from chunks_df.csv
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/chunks_df.csv')

In [ ]:
# Display the DataFrame with embeddings
df

,Unnamed: 0,text,section,article,number1,number2,number3,number4,number5,number6,CELEX number,embedding
0,0,Avis juridique important,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[ -0.71743995 -2.661333 -0.5654966 2.04...
1,1,Special Report No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.5942285e+00 -1.9066961e+00 1.9812379e+00 ...
2,2,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-2.76330781e+00 -3.82428586e-01 -1.05377293e+...
3,3,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.6374424 -2.093941 2.3462052 0.805499...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
...,...,...,...,...,...,...,...,...,...,...,...,...
29602,29602,For the High Authority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-3.7505999 -2.234225 0.9126675 2.350292...
29603,29603,The President,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.93085 -0.78436 2.562405 2.02705 ...
29604,29604,Dino DEL BO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.2211666 -2.5205266 2.42058 -2.599566...
29605,29605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

spacy.prefer_gpu()

# Load a pre-trained English model
nlp = spacy.load("en_core_web_sm")

# Read data from chunks_df.csv
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/chunks_df.csv')

# Define functions for query preprocessing
def preprocess_query(query):
    # Tokenize the query
    doc = nlp(query)
    # Lemmatize tokens and remove stop words
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens)


# Define function for document preprocessing
def preprocess_documents(documents):
    # Tokenize and lemmatize each document
    preprocessed_documents = []
    for doc in nlp.pipe(documents.astype(str)):  # Convert to string
        tokens = [token.lemma_ for token in doc if not token.is_stop]
        preprocessed_documents.append(" ".join(tokens))
    return preprocessed_documents


# Preprocess documents
preprocessed_docs = preprocess_documents(df['text'])

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform preprocessed documents
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_docs)

# Function to retrieve relevant documents for a query
def retrieve_documents(query, tfidf_matrix, tfidf_vectorizer, top_n=5):
    # Preprocess query
    preprocessed_query = preprocess_query(query)
    # Transform preprocessed query using the fitted TF-IDF vectorizer
    query_vector = tfidf_vectorizer.transform([preprocessed_query])
    # Calculate cosine similarity between query vector and document vectors
    similarities = cosine_similarity(query_vector, tfidf_matrix)
    # Get indices of top n similar documents
    top_indices = similarities.argsort()[0][-top_n:][::-1]
    # Return relevant documents
    return df.iloc[top_indices]



In [ ]:
# Example usage
query = "What is the impact of climate change on biodiversity?"
relevant_docs = retrieve_documents(query, tfidf_matrix, tfidf_vectorizer)
print(relevant_docs)


       Unnamed: 0                                               text  \
8248         8248  There is a correlation between: Reporting elem...   
5766         5766  ‘climate adaptation’ means a process that ensu...   
8245         8245  Estimated impact of the production or use of b...   
5908         5908  Within the same time limit, the European Scien...   
12983       12983  anticipated changes of climate (for example, g...   

         section  article  number1 number2 number3 number4 number5  number6  \
8248   ANNEX XVI      NaN      NaN     NaN     NaN     NaN     NaN      NaN   
5766    Document      2.0      NaN    (19)     NaN     NaN     NaN      NaN   
8245   ANNEX XVI      NaN      NaN     NaN     NaN     NaN     NaN      NaN   
5908    Document     12.0      5.0     NaN     NaN     NaN     NaN      NaN   
12983   ANNEX IV      NaN      1.0     NaN     NaN     NaN     NaN      NaN   

      CELEX number                                          embedding  
8248    32022R2299  

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install langchain

In [ ]:
!pip install torch torchvision torchaudio torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!git clone https://github.com/facebookresearch/LLAMA.git

Cloning into 'LLAMA'...
remote: Enumerating objects: 434, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 434 (delta 4), reused 12 (delta 3), pack-reused 417
Receiving objects: 100% (434/434), 1.10 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [ ]:
%cd LLAMA

/content/LLAMA/LLAMA


In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget https://dl.fbaipublicfiles.com/LLAMA/scripts/download.sh
!chmod +x download.sh
!./download.sh


--2024-03-02 10:13:11--  https://dl.fbaipublicfiles.com/LLAMA/scripts/download.sh
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.48, 18.173.166.31, 18.173.166.74, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.48|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-03-02 10:13:11 ERROR 403: Forbidden.

Enter the URL from email: ^C


In [ ]:
'''
import numpy as np
import faiss
import spacy

# Load Spacy model
nlp = spacy.load("en_core_web_md")

# Load Faiss index
index = faiss.read_index('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index')

# Load embeddings array
embeddings_array = np.load('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy')

query_embedding1 = nlp("the Agreement on Partnership and Cooperation was signed by the European Communities and their Members States and Kazakhstan").vector

# Add a batch dimension to the query embedding
query_embedding1 = np.expand_dims(query_embedding1, axis=0)

# Perform the search
D, I = index.search(query_embedding1, k=200)
'''


'\nimport numpy as np\nimport faiss\nimport spacy\n\n# Load Spacy model\nnlp = spacy.load("en_core_web_md")\n\n# Load Faiss index\nindex = faiss.read_index(\'/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index\')\n\n# Load embeddings array\nembeddings_array = np.load(\'/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy\')\n\nquery_embedding1 = nlp("the Agreement on Partnership and Cooperation was signed by the European Communities and their Members States and Kazakhstan").vector\n\n# Add a batch dimension to the query embedding\nquery_embedding1 = np.expand_dims(query_embedding1, axis=0)\n\n# Perform the search\nD, I = index.search(query_embedding1, k=200)\n'

In [ ]:
I

NameError: name 'I' is not defined

In [ ]:
# Extract the corresponding text from the DataFrame
#similar_texts = df.iloc[I[0]]['text'].tolist()

#print(similar_texts)

In [ ]:
#print("Query Embedding Dimension:", query_embedding1.shape)
#print("Index Embedding Dimension:", embeddings_array.shape[1])


In [ ]:
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
#%pip install xformers
%pip install bitsandbytes
%pip install matplotlib seaborn tqdm
%pip install evaluate
%pip install rouge_score
%pip install bert_score

In [ ]:
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
from transformers import LlamaTokenizer
hf_auth = os.environ.get('HF_AUTH')
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",use_auth_token=hf_auth)


In [ ]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'meta-llama/Llama-2-13b-chat-hf'

In [ ]:
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


In [ ]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=os.environ.get('HF_AUTH')
)

In [ ]:
#Loading the model will take some time, (roughly 5 min)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token=os.environ.get('HF_AUTH')
)

model.eval()# we only use the model for inference
print(f"Model loaded ")

In [ ]:
model.get_memory_footprint()

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    # we pass model parameters here too
    temperature=0.01,
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
### your code ###
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)
### your code ###

In [ ]:
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS

# Create a FaissIndexStore
vectorstore = FAISS(query_embedding, index, df['text'],1)


In [ ]:
seen_similarity_scores = set()

# Assuming you have similarity_scores and top_matches_indices
for i, index in enumerate(min_similarity_score_indices, start=1):
    similarity_score = similarity_scores[index]

    # Check if this similarity score has been seen before
    if similarity_score not in seen_similarity_scores:
        seen_similarity_scores.add(similarity_score)

        related_text = df['text'].iloc[index]

        # Generate text based on the related text
        generated_text = generate_text(related_text)

        # Print the results
        print(f"{i}. Similarity Score: {similarity_score}")
        print(f"   Original Text: {related_text}")
        print(f"   Generated Text: {generated_text}")
        print()
